In [2]:
import requests
import json
import urllib.request
import pandas as pd
import ast

In [82]:
response_details_merge = pd.read_csv('../data/Data 6 - Merged Responses and Movie Details.csv')
response_details_merge.head()

,Unnamed: 0,id_x,person_mbti,mbti_test,character,movie,new_title,genre_ids,id_y,popularity,release_date,title,vote_average,vote_count
0,0,0,ENFP,michaelcaloz.com,Tony Stark,Avengers: Endgame,Avengers: Endgame,"[12, 878, 28]",299534,240.886,2019-04-24,Avengers: Endgame,8.255,24867
1,1,0,ENFP,michaelcaloz.com,Rocket Raccoon,Guardians of the Galaxy Vol. 3,Guardians of the Galaxy Vol. 3,"[878, 12, 28]",447365,169.896,2023-05-03,Guardians of the Galaxy Vol. 3,7.972,6262
2,2,0,ENFP,michaelcaloz.com,Captain America,Avengers: Endgame,Avengers: Endgame,"[12, 878, 28]",299534,240.886,2019-04-24,Avengers: Endgame,8.255,24867
3,3,0,ENFP,michaelcaloz.com,Peter Parker,Spider-Man: Far From Home,Spider-Man: Far From Home,"[28, 12, 878]",429617,124.720,2019-06-28,Spider-Man: Far From Home,7.400,15144
4,4,0,ENFP,michaelcaloz.com,Hobie Brown,Spider-Man: Across the Spider-Verse,Spider-Man: Across the Spider-Verse,"[16, 28, 12, 878]",569094,187.181,2023-05-31,Spider-Man: Across the Spider-Verse,8.366,6265


# Actions
### Columns to be removed
- `Unnamed: 0`
- `movie` and `title` since `new_title` was assigned to hold the actual movie title in Script 4
- `vote_count`
### Renames
- `id_x` as `person_id`
- `new_title` as `movie`
- `id_y` as `movie_id`
- `popularity` as `movie_popularity`
- `vote_average` as `movie_vote_avg`
### Others
- Change `release_date` to only capture year information as `movie_year`


In [83]:
response_details = response_details_merge.copy()

response_details.drop(columns = ['Unnamed: 0', 'person_mbti', 'movie', 'title', 'vote_count'], inplace=True)
response_details.rename(columns = {'id_x':'person_id', 'new_title':'movie', 'id_y':'movie_id', 'popularity':'movie_popularity', 'vote_average':'movie_vote_avg'}, inplace = True)
response_details['movie_year'] = response_details_merge['release_date'].astype(str).str[:4]
response_details.drop(columns = ['release_date'], inplace=True)

response_details

,person_id,mbti_test,character,movie,genre_ids,movie_id,movie_popularity,movie_vote_avg,movie_year
0,0,michaelcaloz.com,Tony Stark,Avengers: Endgame,"[12, 878, 28]",299534,240.886,8.255,2019
1,0,michaelcaloz.com,Rocket Raccoon,Guardians of the Galaxy Vol. 3,"[878, 12, 28]",447365,169.896,7.972,2023
2,0,michaelcaloz.com,Captain America,Avengers: Endgame,"[12, 878, 28]",299534,240.886,8.255,2019
3,0,michaelcaloz.com,Peter Parker,Spider-Man: Far From Home,"[28, 12, 878]",429617,124.720,7.400,2019
4,0,michaelcaloz.com,Hobie Brown,Spider-Man: Across the Spider-Verse,"[16, 28, 12, 878]",569094,187.181,8.366,2023
...,...,...,...,...,...,...,...,...,...
700,123,michaelcaloz.com,Agent Smith,The Matrix,"[28, 878]",603,575.344,8.215,1999
701,123,michaelcaloz.com,Daniel Plainview,There Will Be Blood,[18],7345,36.996,8.084,2007
702,123,michaelcaloz.com,Eraserhead Baby,Eraserhead,"[14, 27]",985,67.775,7.400,1978
703,123,michaelcaloz.com,Patrick Bateman,American Psycho,"[53, 18, 80]",1359,84.339,7.404,2000


In [84]:
char_mbti_df = pd.read_csv('../data/Data 8 - Franchise-expanded Movies from Data 2.csv')
char_mbti_df

,char,movie,mbti,votes
0,Guido Anselmi,8 1/2,INTP,57
1,King Leonidas of Sparta,300,ENTJ,460
2,Artemisia,300,ENTJ,127
3,Xerxes,300,INTJ,103
4,Gorgo,300,INFJ,62
...,...,...,...,...
10718,Gazelle,Zootopia (2016),ESFP,107
10719,Mr Manchas,Zootopia (2016),ISFJ,40
10720,Doug Ramses,Zootopia (2016),INTP,30
10721,Travis,Zootopia (2016),ESTP,27


In [85]:
zero = response_details.iloc[-1]
char = zero['character']
movie = zero['movie']
print(char, movie)

The Duke Fear and Loathing in Las Vegas


In [86]:
char_mbti_df_copy = char_mbti_df.copy()
query_bool = char_mbti_df_copy['char'].str.contains(char) & char_mbti_df_copy['movie'].str.contains(movie)
# query_bool =  query_bool[query_bool.values==True]
char_mbti_df_copy = char_mbti_df_copy[query_bool]
char_mbti_df_copy
# print(query_bool)

,char,movie,mbti,votes


In [87]:
char_mbti_df_copy = char_mbti_df.copy()

failed_index = []
failed_char = []
failed_movie = []

k = len(response_details)
# k = 5

for i in range(k):
    zero = response_details.iloc[i]
    char = zero['character']
    movie = zero['movie']
    # print(char, movie)

    query_bool = char_mbti_df_copy['char'].str.contains(char) & char_mbti_df_copy['movie'].str.contains(movie)
    char_mbti_query = char_mbti_df_copy[query_bool]
    try:
        char_df_subset = char_mbti_query.iloc[0]
        print(i, char_df_subset['char'], char_df_subset['mbti'])
    except:
        print(i, '      FAILED!  ', char, movie)
        failed_index.append(i)
        failed_char.append(char)
        failed_movie.append(movie)

failed_df = pd.DataFrame({'failed_index': failed_index, 'failed_char': failed_char, 'failed_movie': failed_movie})
failed_df

0 Tony Stark Iron Man ENTP
1 Rocket Raccoon ESTP
2 Steve Rogers Captain America ISFJ
3 Peter Parker  Spider-Man ENFP
4 Hobart Hobie Brown Spider-Punk ESTP
5 Peter Parker Spider-Man ISTP
6 Celine INFP
7 Jesse Wallace ENTP
8 Bella Baxter ENTP
9       FAILED!   Miles Morales Spider-Man: Across the Spider-Verse
10 Johnny English ENTP
11 Tony Stark Iron Man ENTP
12       FAILED!   Captain Pete "Maverick" Mitchell Top Gun: Maverick
13 Neo INTP
14       FAILED!   Parzival (Wade Watts) Ready Player One
15       FAILED!   Benoit Blanc Glass Onion: A Knives Out Mystery
16       FAILED!   Dr. Stephen Strange Doctor Strange
17       FAILED!   J. Robert Oppenheimer Oppenheimer
18 Puss in Boots ESTP
19 Loki Laufeyson INFJ
20       FAILED!   Amelie Poulain Amélie
21 V INTJ
22 Ellen Ripley INTJ
23       FAILED!   Mathilda Mathilda


C:\Users\Marco\AppData\Local\Temp\ipykernel_18436\2086524206.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  query_bool = char_mbti_df_copy['char'].str.contains(char) & char_mbti_df_copy['movie'].str.contains(movie)


24       FAILED!   Willy Wonka Willy Wonka & the Chocolate Factory
25       FAILED!   Remus Lupin Harry Potter and the Philosopher's Stone
26       FAILED!   Wall-E WALL·E
27       FAILED!   Waymond Wang Everything Everywhere All at Once
28 Morticia Addams INFJ
29 Wanda Maximoff Scarlet Witch INFP
30 Joy ENFJ
31 Jack Dawson ESFP
32       FAILED!   Grace Le domas Ready or Not
33       FAILED!   Naomi Misora Death Note Relight 1: Visions of a God
34       FAILED!   Yorichii  Demon Slayer: Kimetsu no Yaiba -To the Hashira Training-
35 Amy Elliott Dunne INFJ
36 Charlie Kelmeckis INFP
37 Bruce Wayne Batman INTJ
38 Tony Stark Iron Man ENTP
39       FAILED!   Otto A Man Called Otto
40 Katniss Everdeen ISTP
41 Joy ENFJ
42 Elle Woods ENFJ
43 Gwendolyn Gwen Stacy ENFJ
44       FAILED!   Jang Okju Ballerina
45       FAILED!   Sienna Shaw Terrifier 2
46       FAILED!   Jordan Turner The Call
47 Erin Harson ISTP
48 Gretchen Wieners ESFJ
49 Ken ESFP
50 Miles Morales Spider-Man ISFP
51 Tiana ESTJ
52 

,failed_index,failed_char,failed_movie
0,9,Miles Morales,Spider-Man: Across the Spider-Verse
1,12,"Captain Pete ""Maverick"" Mitchell",Top Gun: Maverick
2,14,Parzival (Wade Watts),Ready Player One
3,15,Benoit Blanc,Glass Onion: A Knives Out Mystery
4,16,Dr. Stephen Strange,Doctor Strange
...,...,...,...
320,696,Kiri,Avatar: The Way of Water
321,697,Terminator (Arnold Schwarzenegger),Terminator 3: Rise of the Machines
322,699,Uta,One Piece Film Red
323,702,Eraserhead Baby,Eraserhead
